In [ ]:
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install scikit-learn
# !pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [ ]:
# Load dataset from local file
df = pd.read_csv('/content/1_boston_housing.csv')

In [ ]:
# Split features and target variable
X = df.loc[:, df.columns != 'MEDV']
y = df.loc[:, df.columns == 'MEDV']

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [ ]:
# Feature scaling using MinMaxScaler
mms = MinMaxScaler()
mms.fit(X_train)
X_train = mms.transform(X_train)
X_test = mms.transform(X_test)

In [ ]:
# Build the neural network model
model = Sequential()
model.add(Dense(128, input_shape=(13,), activation='relu', name='dense_1'))
model.add(Dense(64, activation='relu', name='dense_2'))
model.add(Dense(1, activation='linear', name='dense_output'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
# Summary of the model
model.summary()

In [ ]:
# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
# Model checkpoint callback to save the best model based on validation loss
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
# Train the model with validation split and callbacks
history = model.fit(X_train, y_train, epochs=100, validation_split=0.05, verbose=1,
                    callbacks=[early_stopping, checkpoint])

In [ ]:
# Evaluate the model on the test set
mse_nn, mae_nn = model.evaluate(X_test, y_test)

print('Mean squared error on test data: ', mse_nn)
print('Mean absolute error on test data: ', mae_nn)

In [ ]:
# Predict on test data
y_pred = model.predict(X_test)

In [ ]:
# Convert y_test and y_pred to 1D arrays for comparison
y_test_flat = y_test.values.flatten()
y_pred_flat = y_pred.flatten()

In [ ]:
# Print first 10 predictions vs actual values
print("\nFirst 10 Predictions vs Actual:")
for true, pred in zip(y_test_flat[:10], y_pred_flat[:10]):
    print(f"Actual: {float(true):.2f}, Predicted: {float(pred):.2f}")

In [ ]:
# Compute R² score
r2 = r2_score(y_test_flat, y_pred_flat)
print(f"\nR² Score on Test Data: {r2:.4f}")

In [ ]:
# Plot Actual vs Predicted House Prices
plt.figure(figsize=(8,6))
plt.scatter(y_test_flat, y_pred_flat, alpha=0.7)
plt.plot([y_test_flat.min(), y_test_flat.max()], [y_test_flat.min(), y_test_flat.max()], 'r--')
plt.xlabel('Actual MEDV')
plt.ylabel('Predicted MEDV')
plt.title('Actual vs Predicted House Prices')
plt.grid(True)
plt.show()

In [ ]:
# Plot Residuals vs Predicted House Prices
residuals = y_test_flat - y_pred_flat
plt.figure(figsize=(8,6))
plt.scatter(y_pred_flat, residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted MEDV')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted House Prices')
plt.grid(True)
plt.show()